# 1. 데이터 집계  
- groupby 메서드로 평균값 구하기

In [2]:
import pandas as pd

df = pd.read_csv('doit_pandas-master/data/gapminder.tsv',sep='\t')

In [3]:
# year을 기준으로 그룹화하고 lifeExp 열의 평균을 구함
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


- 분할-반영-결합 과정 살펴보기 - groupby 메서드

In [4]:
# 분할
# year 열의 데이터를 중복없이 추출
years = df.year.unique()
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [5]:
# 반영
# 연도별 평균값 구하기
# 다음과정을 반복
y1952 = df.loc[df.year == 1952, :]
y1952_mean = y1952.lifeExp.mean()
print(y1952_mean)

49.05761971830987


In [6]:
# 결합
# 연도별 lifeExp 평균값 합치기
# df2 = pd.DataFrame({"year": [1952,1957,1962,2007]}, "":[y1952_mean,y1957_mean,y1962_mean,y2007_mean]})

- 집계 메서드   
    - count : 누락값을 제외한 데이터 수  
    - size : 누락값을 포함한 데이터 수  
    - mean : 평균값  
    - std : 표준편차  
    - min : 최솟값  
    - quantile(q=0.25) : 백분위 수 25%  
    - max : 최대값  
    - sum : 전체 합  
    - var : 분산  
    - sem : 평균의 표준편차   
    - describe : 데이터 수, 평균, 표준편차, 최소값, 백분위수, 최대값  
    - first : 첫번째 행 
    - last : 마지막 행
    - nth : n번째 행

- agg 메서드로 사용자 함수와 groupby 메서드 조합하기

In [10]:
# 평균값을 구하는 사용자함수
# 입력받은 열의 평균값
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    return sum / n

In [12]:
# agg메서드 사용해 groupby 메서드와 조합
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [13]:
# 2개의 인자값을 받아 처리하는 사용자 함수
# 첫번째 열의 평균값을 구하여 두번째 값과의 차이를 계산
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum/n
    return mean-diff_value

In [14]:
# lifeExp 평균
global_mean = df.lifeExp.mean()
print(global_mean)

# 연도별 평균수명 - 전체 평균수명
# agg메서드 활용
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value=global_mean)
print(agg_mean_diff)

59.47443936619713
year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


- 여러개의 집계 메서드 한번에 사용하기

In [15]:
# lifeExp 열의 0이 아닌 값의 개수, 평균, 표준편차 
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
print(gdf)

      count_nonzero       mean        std
year                                     
1952          142.0  49.057620  12.225956
1957          142.0  51.507401  12.231286
1962          142.0  53.609249  12.097245
1967          142.0  55.678290  11.718858
1972          142.0  57.647386  11.381953
1977          142.0  59.570157  11.227229
1982          142.0  61.533197  10.770618
1987          142.0  63.212613  10.556285
1992          142.0  64.160338  11.227380
1997          142.0  65.014676  11.559439
2002          142.0  65.694923  12.279823
2007          142.0  67.007423  12.073021


In [18]:
# 집계 메서드를 딕셔너리에 담아 agg메서드에 전달
gdf_dict = df.groupby('year').agg({'lifeExp':'mean', 'pop':'median', 'gdpPercap':'median'})
print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334741
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371109


# 2. 데이터 변환  
- 표준편차 계산

In [20]:
# 표준점수 계산
def my_zscore(x):
    return (x-x.mean()) / x.std()

In [22]:
# transform 메서드 사용해 함수 적용
transform_z = df.groupby('year').lifeExp.transform(my_zscore)
print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [23]:
# 데이터를 표준화할 뿐 집계하지 않는다
# 데이터의 양이 줄어들지 않는다
print(df.shape)
print(transform_z.shape)

(1704, 6)
(1704,)


- 누락값을 평균값으로 처리하기

In [27]:
# 임의의 4개의 값을 누락값으로 바꾸기
import pandas as pd
import numpy as np
import seaborn as sns
np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4],'total_bill'] = np.NaN
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [29]:
# 성별 별 데이터 수
count_sex = tips_10.groupby('sex').count()
print(count_sex)

        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3


In [32]:
# 성별에 따른 total_bill 평균값을 누락값에 넣기
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [34]:
# 새로운 열에 누락값을 채운 값 추가
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00    Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16    Male    Yes   Sat  Dinner     4          17.9525
198         NaN  2.00  Female    Yes  Thur   Lunch     2          13.9300
176         NaN  2.00    Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56    Male    Yes  Thur   Lunch     2          28.4400
124       12.48  2.52  Female     No  Thur   Lunch     2          12.4800
9         14.78  3.23    Male     No   Sun  Dinner     2          14.7800
101       15.38  3.00  Female    Yes   Fri  Dinner     2          15.3800


# 3. 데이터 필터링  
- 데이터 필터링 사용하기 : filter메서드

In [35]:
tips = sns.load_dataset('tips')
print(tips.shape)

(244, 7)


In [38]:
print(tips['size'].value_counts())

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64


In [40]:
# 5,6,1 테이블의 주문이 매우 적기 때문에 제외시킨다
# 30번이상의 주문이 있는 테이블만 그룹화하여 저장
tips_filtered = tips.\
    groupby('size').\
    filter(lambda x:x['size'].count()>=30)
print(tips_filtered['size'].value_counts())

2    156
3     38
4     37
Name: size, dtype: int64


# 4. 그룹 오브젝트

- 그룹 오브젝트 저장하여 살펴보기

In [42]:
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [44]:
# groupby 메서드의 결과값의 자료형은 그룹 오브젝트
grouped = tips_10.groupby('sex')
print(grouped)

In [45]:
# 그룹 오브젝트에 포함된 그룹 보기 groups 속성
print(grouped.groups)

{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}


- 그룹 오브젝트의 평균 구하기

In [46]:
avgs = grouped.mean()
print(avgs)

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


In [47]:
# 평균값을 계산할 수 없는 열은 자동제외됨
print(tips_10.columns)

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')
